Import

In [1]:
import subprocess
import shlex
import os.path
import sys
import pickle
import datetime
import tweepy
import matplotlib.pyplot as plt 
import numpy as np
from scipy import spatial  
from sklearn.metrics import r2_score
import string
import numpy as np
import matplotlib.pyplot as plt
import collections
from itertools import tee, islice
from sklearn.decomposition import PCA
import math

Other Helper Functions

In [2]:
def get_list_of_users(tweets):

    list_of_user = []

    for tweet in tweets:
        user_id = tweet.user.id
        if user_id not in list_of_user:
            list_of_user.append(user_id)

    return list_of_user

Import Tweet and Related Data

In [ ]:
# Where you want to save the coverted files # TO BE REMOVED
test_directory = "" 

In [3]:
#Location of all the ".p" files containing tweets of friends
directory = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/" 

# Location of sentiment analysis file
sentiment_directory = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/Sentiment/"
analyzed_users = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/data_analyzed_1.p"

In [4]:
# Tweets output from the function, get_inital_tweets
initial_tweet_file = directory + 'user_tweet_large.p'
initial_tweets = pickle.load(open(initial_tweet_file, "rb"))
trending_tweet_file = directory + 'trending_tweets.p'
trending_tweets = pickle.load(open(trending_tweet_file, "rb"))

In [3]:
# Don't use this if using above
already_analyzed = pickle.load(open(analyzed_users, "rb"))
list_of_users = []

for uid in already_analyzed:
    list_of_users.append(uid)

Prototype and Exemplar Models - Helper Functions

In [5]:
# Based on code from COG260
# Helper method for similarity function
def similarity_function(feature_vector_0, feature_vector_1):
    euclidean_distance = spatial.distance.euclidean(feature_vector_0, feature_vector_1)
    temp_calc = euclidean_distance**2
    similarity = np.exp(0 - temp_calc)
    return similarity
    
    
# Helper method for examplar model
def categorize_with_exemplar(test_data_point, category_0, category_1, category_2):
    
    similarity_sum_cateogry_0 = []
    similarity_sum_cateogry_1 = []
    similarity_sum_cateogry_2 = []
    
    for object_0 in category_0:
        similarity_sum_cateogry_0.append(similarity_function(object_0, test_data_point))
        
    for object_1 in category_1:
        similarity_sum_cateogry_1.append(similarity_function(object_1, test_data_point))
        
    for object_2 in category_2:
        similarity_sum_cateogry_2.append(similarity_function(object_2, test_data_point))
        
    similarity_sum_cateogry_0 = np.sum(similarity_sum_cateogry_0)
    similarity_sum_cateogry_1 = np.sum(similarity_sum_cateogry_1)
    similarity_sum_cateogry_2 = np.sum(similarity_sum_cateogry_2)
    
    similarity_0 = similarity_sum_cateogry_0 / len(category_0)
    similarity_1 = similarity_sum_cateogry_1 / len(category_1)
    similarity_2 = similarity_sum_cateogry_2 / len(category_2)
    
    combined = [similarity_0, similarity_1, similarity_2]
    return_value = combined.index(max(combined))
    #print(combined)
    #print(return_value)
    
    return return_value

def categorize_with_2_exemplar(test_data_point, category_0, category_1):
    
    similarity_sum_cateogry_0 = []
    similarity_sum_cateogry_1 = []
    
    for object_0 in category_0:
        similarity_sum_cateogry_0.append(similarity_function(object_0, test_data_point))
        
    for object_1 in category_1:
        similarity_sum_cateogry_1.append(similarity_function(object_1, test_data_point))
        
    similarity_sum_cateogry_0 = np.sum(similarity_sum_cateogry_0)
    similarity_sum_cateogry_1 = np.sum(similarity_sum_cateogry_1)
    
    similarity_0 = similarity_sum_cateogry_0 / len(category_0)
    similarity_1 = similarity_sum_cateogry_1 / len(category_1)

    combined = [similarity_0, similarity_1]
    return_value = combined.index(max(combined))
    
    return return_value

In [6]:
# Helper method to get the prototype
def get_prototype(data_point):
        
    prototype = []
        
    for i in range(len(data_point[0, :])):
        prototype.append(np.mean(data_point[:,i]))
            
    return prototype

def categorize_with_prototype(test_data_point, category_0, category_1, category_2):
    
    cat_0_distance = spatial.distance.euclidean(test_data_point, category_0)
    cat_1_distance = spatial.distance.euclidean(test_data_point, category_1)
    cat_2_distance = spatial.distance.euclidean(test_data_point, category_2)
    
    combined = [cat_0_distance, cat_1_distance, cat_2_distance]
    return_value = combined.index(min(combined))
    
    return return_value

def categorize_with_2_prototype(test_data_point, category_0, category_1):
    
    cat_0_distance = spatial.distance.euclidean(test_data_point, category_0)
    cat_1_distance = spatial.distance.euclidean(test_data_point, category_1)
    
    combined = [cat_0_distance, cat_1_distance]
    return_value = combined.index(min(combined))
    
    return return_value

In [7]:
def categorize_with_score_prototype(test_data_point, category_0, category_1, category_2, category_3, category_4, category_5, category_6, category_7, category_8):
    
    cat_0_distance = spatial.distance.euclidean(test_data_point, category_0)
    cat_1_distance = spatial.distance.euclidean(test_data_point, category_1)
    cat_2_distance = spatial.distance.euclidean(test_data_point, category_2)
    cat_3_distance = spatial.distance.euclidean(test_data_point, category_3)
    cat_4_distance = spatial.distance.euclidean(test_data_point, category_4)
    cat_5_distance = spatial.distance.euclidean(test_data_point, category_5)
    cat_6_distance = spatial.distance.euclidean(test_data_point, category_6)
    cat_7_distance = spatial.distance.euclidean(test_data_point, category_7)
    cat_8_distance = spatial.distance.euclidean(test_data_point, category_8)
    
    combined = [cat_0_distance, cat_1_distance, cat_2_distance, cat_3_distance, cat_4_distance, cat_5_distance, cat_6_distance, cat_7_distance, cat_8_distance]
    return_value = combined.index(min(combined))
    
    return return_value

Prototype and Exemplar Model - Model Prep

In [9]:
sentiment_directory = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/Sentiment/"
analyzed_users = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/data_analyzed_1.p"
analyzed_users = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/data_analyzed_1.p"


data_point_n_4 = []
data_point_n_3 = []
data_point_n_2 = []
data_point_n_1 = []
data_point_n_0 = []
data_point_p_1 = []
data_point_p_2 = []
data_point_p_3 = []
data_point_p_4 = []

already_analyzed = pickle.load(open(analyzed_users, "rb"))
list_of_users = []

for uid in already_analyzed:
    list_of_users.append(uid)

# Analyze all the  users in initial_tweets
for user in list_of_users:
    
    file_name = sentiment_directory + str(user) + '_sent.p'
    
    if os.path.exists(file_name):
        
        tweet_sentiment = pickle.load(open(file_name, "rb"))
        
        for tweet_id in tweet_sentiment:
            positive = 0
            negative = 0
            neutral = 0
            contain_pos = 0
            contain_neg = 0
            
            tweet_sent = tweet_sentiment[tweet_id]['combined']

            if 'friend' in tweet_sentiment[tweet_id]:
                for tweet in tweet_sentiment[tweet_id]['friend']:
                    friend_sentiment = tweet_sentiment[tweet_id]['friend'][tweet]['combined']
                    
                    if friend_sentiment > 0:
                        positive += 1
                    elif friend_sentiment < 0:
                        negative += 1
                    else:
                        neutral += 1
                        
                    friend_sentiment_pos = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][0]
                    friend_sentiment_neg = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][1]
                    
                    if friend_sentiment_pos > 1:
                        contain_pos += 1
                        
                    if friend_sentiment_neg < -1:
                        contain_neg += 1
                            
                total = positive + negative + neutral
                
                if total > 0:
                    positive_percent = positive/total
                    negative_percent = negative/total
                    neutral_percent = neutral/total
                    
                    cont_pos_percent = contain_pos/total
                    cont_neg_percent = contain_neg/total
                    
                    temp = [positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent]
                    temp = np.asarray(temp)

                    if tweet_sent == -4:
                        data_point_n_4.append(temp)
                    elif tweet_sent == -3:
                        data_point_n_3.append(temp)
                    elif tweet_sent == -2:
                        data_point_n_2.append(temp)
                    elif tweet_sent == -1:
                        data_point_n_1.append(temp)
                    elif tweet_sent == 0:
                        data_point_n_0.append(temp)
                    elif tweet_sent == 1:
                        data_point_p_1.append(temp)
                    elif tweet_sent == 2:
                        data_point_p_2.append(temp)
                    elif tweet_sent == 3:
                        data_point_p_3.append(temp)
                    elif tweet_sent == 4:
                        data_point_p_4.append(temp)

data_point_n_4 = np.asarray(data_point_n_4)
data_point_n_3 = np.asarray(data_point_n_3)
data_point_n_2 = np.asarray(data_point_n_2)
data_point_n_1 = np.asarray(data_point_n_1)
data_point_n_0 = np.asarray(data_point_n_0)
data_point_p_1 = np.asarray(data_point_p_1)
data_point_p_2 = np.asarray(data_point_p_2)
data_point_p_3 = np.asarray(data_point_p_3)
data_point_p_4 = np.asarray(data_point_p_4)

Prototype Model

In [10]:
# Get Prototype
# Prototype chosen due to various studies that suggest proportion of post and whether 
# you have seen negativity 
#(i.e. not the overall sentiment, but whether there is any hint of negativity)

prototype_n_1 = get_prototype(data_point_n_1)
prototype_n_2 = get_prototype(data_point_n_2)
prototype_n_3 = get_prototype(data_point_n_3)
prototype_n_4 = get_prototype(data_point_n_4)
prototype_n_0 = get_prototype(data_point_n_0)
prototype_p_1 = get_prototype(data_point_p_1)
prototype_p_2 = get_prototype(data_point_p_2)
prototype_p_3 = get_prototype(data_point_p_3)
prototype_p_4 = get_prototype(data_point_p_4)

In [15]:
print(prototype_n_4)
print(prototype_n_3)
print(prototype_n_2)
print(prototype_n_1)
print(prototype_n_0)
print(prototype_p_1)
print(prototype_p_2)
print(prototype_p_3)
print(prototype_p_4)

[0.5126910284070478, 0.12110319172616905, 0.3662057798667831, 0.0, 1.0]
[0.3452308270471499, 0.14878076821321132, 0.5059884047396388, 0.022900763358778626, 1.0]
[0.3378288067171415, 0.15970149014067128, 0.5024697031421872, 0.22821576763485477, 1.0]
[0.3323557406348522, 0.15657513454269747, 0.5110691248224503, 0.1813953488372093, 1.0]
[0.33347087874083997, 0.1481407023076638, 0.5183884189514961, 0.07017543859649122, 0.07017543859649122]
[0.3445961375149226, 0.15148264489754074, 0.5039212175875366, 1.0, 0.06029285099052541]
[0.3438294305887958, 0.1627417230819233, 0.4934288463292809, 1.0, 0.03579952267303103]
[0.35276381460033973, 0.16165904346989957, 0.4855771419297607, 1.0, 0.0]
[0.26724633140111753, 0.060264243563632566, 0.6724894250352498, 1.0, 0.0]


In [11]:
# Prep for Testing Model
result = {}

NZ_tweet_dir = predictive_model = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Partner Code/"
predictive_model = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Partner Code/Sentiment/"

#tweet_file_name = predictive_model  + 'user_tweet_large_NZ_1.p'
#predictive_user_tweets = pickle.load(open(tweet_file_name, "rb"))

trending_tweet_file_NZ = NZ_tweet_dir + 'trending_tweets_NZ.p'
trending_tweets_NZ = pickle.load(open(trending_tweet_file_NZ, "rb"))

list_of_users_NZ = get_list_of_users(trending_tweets_NZ)

analyzed_users_NZ = predictive_model  + 'data_analyzed_1.p'
already_analyzed_NZ = pickle.load(open(analyzed_users_NZ, "rb"))

for uid in already_analyzed_NZ:
    if uid not in list_of_users_NZ:
        list_of_users_NZ.append(uid)
        
print(len(list_of_users_NZ))

191


In [20]:
# Prototype - only positive and negative

total_count = 0
accurate = 0

p_total = 0
p_acc = 0

n_total = 0
n_acc = 0

ne_total = 0
ne_acc = 0

pos_total = 0
neg_total = 0

p_miss = 0
n_miss = 0

positive_total = 0
negative_total = 0

score_index = [-4, -3, -2, -1, 0, 1, 2, 3, 4]
    
# Analyze all the  users in initial_tweets
for user in list_of_users_NZ:
    
    file_name = predictive_model + str(user) + '_sent.p'
    
    if os.path.exists(file_name):
        
        tweet_sentiment = pickle.load(open(file_name, "rb"))
        
        for tweet_id in tweet_sentiment:

            positive = 0
            negative = 0
            neutral = 0
            contain_pos = 0
            contain_neg = 0
            
            tweet_sent = tweet_sentiment[tweet_id]['combined']
            
            if tweet_sent != 0:

                if 'friend' in tweet_sentiment[tweet_id]:
                    for tweet in tweet_sentiment[tweet_id]['friend']:

                        friend_sentiment = tweet_sentiment[tweet_id]['friend'][tweet]['combined']
                        if friend_sentiment > 0:
                            positive += 1
                        elif friend_sentiment < 0:
                            negative += 1
                        else:
                            neutral += 1
                            
                        friend_sentiment_pos = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][0]
                        friend_sentiment_neg = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][1]
                    
                        if friend_sentiment_pos > 1:
                            contain_pos += 1
                        
                        if friend_sentiment_neg < -1:
                            contain_neg += 1
                            

                total = positive + negative + neutral
                
                if total > 0:
                    positve_percent = positive/total
                    negative_percent = negative/total
                    neutral_percent = neutral/total
                    
                    cont_pos_percent = contain_pos/total
                    cont_neg_percent = contain_neg/total
                    
                    temp = [positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent]
                    temp = np.asarray(temp)

                    exemplar = categorize_with_score_prototype(temp, prototype_n_4, prototype_n_3, prototype_n_2, prototype_n_1, prototype_n_0, prototype_p_1, prototype_p_2, prototype_p_3, prototype_p_4) 
                    exemplar = score_index[exemplar]
                    total_count += 1
                    
                    
                    #print(exemplar, tweet_sent)
                    if exemplar == tweet_sent:
                        accurate += 1
                        

In [21]:
print(accurate)
print(total_count)

279
2581
